In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [35]:
class Clustering:
    def __init__(self, path='accounts_style.csv' ):
        self.path = path
        self.data = self.__preprecessing(path)
        self.data = self.__normalize_based_on_values([2, 2, 1.5, 1.25, 1.75])
        print(self.data)


    def 



    def __preprecessing(self, path):
        #select special part of the dataframe
        data = pd.read_csv(path)
        data = data.iloc[ : ,2:]

        #spilit data and add new culomns
        data[['Age1', 'Age2', 'Age3']] = data["Age"].str.split(",", n=2, expand=True)
        data[['Color1', 'Color2', 'Color3']] = data["Color"].str.split(",", n=2, expand=True)
        data[['Pattern1', 'Pattern2', 'Pattern3']] = data["Pattern"].str.split(",", n=2, expand=True)
        data[['Size1', 'Size2', 'Size3']] = data["Size"].str.split(",", n=2, expand=True)
        data[['Formal1', 'Formal2', 'Formal3']] = data["Formal"].str.split(",", n=2, expand=True)

        #drop unusage columns
        data.drop(columns =['Age', 'Color', 'Pattern', 'Size', 'Formal'], inplace = True)

        #convert type from str to float64
        data = data.astype('float64')

        #calculate the average of each row specialy
        data['Age_avg'] = self.__special_mean(data[['Age1', 'Age2', 'Age3']].values,20)
        data['Color-avg'] = self.__special_mean(data[['Color1', 'Color2', 'Color3']].values,20)
        data['Pattern_avg'] = self.__special_mean(data[['Pattern1', 'Pattern2', 'Pattern3']].values,20)
        data['Size-avg'] = self.__special_mean(data[['Size1', 'Size2', 'Size3']].values,20)
        data['Formal_avg'] = self.__special_mean(data[['Formal1', 'Formal2', 'Formal3']].values,20)

        #drop unusage columns
        data.drop(columns =['Age1', 'Age2', 'Age3', 'Color1', 'Color2', 'Color3', 'Pattern1',
         'Pattern2', 'Pattern3', 'Size1', 'Size2', 'Size3', 'Formal1', 'Formal2', 'Formal3'], inplace = True)

        return data



    def __normalize_based_on_values(self, values):
        #normalize data from 0 to 1
        normalized_data = pd.DataFrame(preprocessing.normalize(self.data), columns=data.columns)

        #put it in -1 to +1
        normalized_data = (normalized_data - 0.5) * 2

        #change the period for each component to change the influence
        normalized_data = normalized_data * values
        return data


    def __special_mean(self, selectedClothes, anomaly):
        # we have to ignore anomaly values in each component to have accessed the real mean value
        meanVector = []
        for i in range(np.shape(selectedClothes)[0]):
            componentSum = 0
            count = 0
            for j in range(np.shape(selectedClothes)[1]):
                if(selectedClothes[i][j] != anomaly):
                    componentSum += selectedClothes[i][j]
                    count += 1
            meanVector.append(componentSum/count)

        return meanVector




In [36]:
clustering = Clustering()


      Age_avg  Color-avg  Pattern_avg  Size-avg  Formal_avg
0   -0.351673   0.747211    -1.087918 -0.219796   -0.307714
1   -0.045289   0.443389    -1.133492 -0.333729   -0.039628
2   -0.845299   0.886751    -1.066987 -0.167468   -0.234456
3   -0.780011   1.049971    -1.042504 -0.487507   -0.148765
4   -0.718975   0.562050    -1.019616 -0.049039   -0.068654
..        ...        ...          ...       ...         ...
395 -0.166603  -0.166603    -0.949981 -0.333302    0.495911
396 -0.592805   0.814390    -0.233524 -0.546402   -0.518704
397 -0.492443   0.713602    -0.143199 -0.307777   -0.958533
398 -0.158851  -0.158851     0.157034 -0.329425   -0.461196
399 -0.623143   0.753714    -0.260829 -0.389465   -0.545250

[400 rows x 5 columns]


In [29]:

data = pd.read_csv('accounts_style.csv')
data = data.iloc[ : ,2:]

#spilit data and add new culomns
data[['Age1', 'Age2', 'Age3']] = data["Age"].str.split(",", n=2, expand=True)
data[['Color1', 'Color2', 'Color3']] = data["Color"].str.split(",", n=2, expand=True)
data[['Pattern1', 'Pattern2', 'Pattern3']] = data["Pattern"].str.split(",", n=2, expand=True)
data[['Size1', 'Size2', 'Size3']] = data["Size"].str.split(",", n=2, expand=True)
data[['Formal1', 'Formal2', 'Formal3']] = data["Formal"].str.split(",", n=2, expand=True)

#drop unusage columns
data.drop(columns =['Age', 'Color', 'Pattern', 'Size', 'Formal'], inplace = True)

#convert type from str to float64
data = data.astype('float64')

#calculate the average of each row specialy
data['Age_avg'] = __special_mean(data[['Age1', 'Age2', 'Age3']].values,20)
data['Color-avg'] = __special_mean(data[['Color1', 'Color2', 'Color3']].values,20)
data['Pattern_avg'] = __special_mean(data[['Pattern1', 'Pattern2', 'Pattern3']].values,20)
data['Size-avg'] = __special_mean(data[['Size1', 'Size2', 'Size3']].values,20)
data['Formal_avg'] = __special_mean(data[['Formal1', 'Formal2', 'Formal3']].values,20)

#drop unusage columns
data.drop(columns =['Age1', 'Age2', 'Age3', 'Color1', 'Color2', 'Color3', 'Pattern1',
'Pattern2', 'Pattern3', 'Size1', 'Size2', 'Size3', 'Formal1', 'Formal2', 'Formal3'], inplace = True)
         
         

data = pd.DataFrame(preprocessing.normalize(data), columns=data.columns)
data = (data - 0.5) * 2
data = data * [2, 2, 1.5, 1.25, 1.75]


     Age_avg  Color-avg  Pattern_avg  Size-avg  Formal_avg
0        3.0        5.0          1.0       3.0         3.0
1        4.0        5.0          1.0       3.0         4.0
2        2.0        5.0          1.0       3.0         3.0
3        2.0        5.0          1.0       2.0         3.0
4        2.0        4.0          1.0       3.0         3.0
..       ...        ...          ...       ...         ...
395      2.5        2.5          1.0       2.0         3.5
396      2.5        5.0          3.0       2.0         2.5
397      2.5        4.5          3.0       2.5         1.5
398      2.5        2.5          3.0       2.0         2.0
399      2.5        5.0          3.0       2.5         2.5

[400 rows x 5 columns]
      Age_avg  Color-avg  Pattern_avg  Size-avg  Formal_avg
0   -0.175837   0.373606    -0.725279 -0.175837   -0.175837
1   -0.022644   0.221694    -0.755661 -0.266983   -0.022644
2   -0.422650   0.443376    -0.711325 -0.133975   -0.133975
3   -0.390006   0.524986    

,Age_avg,Color-avg,Pattern_avg,Size-avg,Formal_avg
0,-0.351673,0.747211,-1.087918,-0.219796,-0.307714
1,-0.045289,0.443389,-1.133492,-0.333729,-0.039628
2,-0.845299,0.886751,-1.066987,-0.167468,-0.234456
3,-0.780011,1.049971,-1.042504,-0.487507,-0.148765
4,-0.718975,0.562050,-1.019616,-0.049039,-0.068654
...,...,...,...,...,...
395,-0.166603,-0.166603,-0.949981,-0.333302,0.495911
396,-0.592805,0.814390,-0.233524,-0.546402,-0.518704
397,-0.492443,0.713602,-0.143199,-0.307777,-0.958533
398,-0.158851,-0.158851,0.157034,-0.329425,-0.461196


In [15]:
def __special_mean(selectedClothes, anomaly):
        # we have to ignore anomaly values in each component to have accessed the real mean value
        meanVector = []
        for i in range(np.shape(selectedClothes)[0]):
            componentSum = 0
            count = 0
            for j in range(np.shape(selectedClothes)[1]):
                if(selectedClothes[i][j] != anomaly):
                    componentSum += selectedClothes[i][j]
                    count += 1
            meanVector.append(componentSum/count)

        return meanVector